In [4]:
import pygame
import random
import math
import time
import queue
import os
import boto3
import pandas as pd
from io import StringIO
from datetime import datetime, timedelta, date

pygame 2.6.1 (SDL 2.28.4, Python 3.12.4)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [8]:
'''# Create a session using your configured AWS credentials
s3 = boto3.client('s3')

# Upload a file to S3
bucket_name = "trackage.1"
file_name = "luggage_dataset.csv" # Path to your local file
s3_object_name = "database.csv" # Name in S3

s3.upload_file(file_name, bucket_name, s3_object_name)

print(f"Uploaded {file_name} to S3 bucket {bucket_name} as {s3_object_name}")


# Download the file from S3
obj = s3.get_object(Bucket=bucket_name, Key=s3_object_name)
'''
# Read it as a Pandas DataFrame
df = pd.read_csv('/Users/layal/Desktop/luggage_dataset5.csv')
df = df[df['Cluster_Size'] != 1]
#df = df[df['Claim_Option'] == 'SLHS Electronic Gates']

print(df.head()) # Show first 5 rows

# Save the updated DataFrame to a temporary local file
temp_file = '/tmp/luggage_dataset5.csv' # Local path to save the modified file
df.to_csv(temp_file, index=False)
'''
# Upload the updated file back to the same location on S3 (this will overwrite the file)
s3.upload_file(temp_file, bucket_name, s3_object_name)

print(f"File {s3_object_name} updated successfully in S3.")'''

              Bag_ID           Claim_Option  Handled_by_SLHS Reservation_ID  \
0  001-AA3472-000001  SLHS Electronic Gates             True      RES-00001   
1  001-AA3472-000002  SLHS Electronic Gates             True      RES-00001   
2  001-AA3472-000003  SLHS Electronic Gates             True      RES-00001   
3  001-AA3472-000004  SLHS Electronic Gates             True      RES-00001   
4  001-AA3472-000009  SLHS Electronic Gates             True      RES-00004   

   Phone_Number Last_Name  Password Flight_Number Flight_Arrival_Time  \
0   15783769457     Wyatt  CX\1eqm2        AA3472               12:44   
1   15783769457     Wyatt  CX\1eqm2        AA3472               12:44   
2   15783769457     Wyatt  CX\1eqm2        AA3472               12:44   
3   15783769457     Wyatt  CX\1eqm2        AA3472               12:44   
4   14106045963   Johnson  B0D064p!        AA3472               12:44   

   Cluster_Size  Delivery_Address Estimated_Arrival_Time Actual_Arrival_Time  \
0     

'\n# Upload the updated file back to the same location on S3 (this will overwrite the file)\ns3.upload_file(temp_file, bucket_name, s3_object_name)\n\nprint(f"File {s3_object_name} updated successfully in S3.")'

In [10]:
conveyor_belts = [
    [(1200, 600), (100, 600)],
    [(100, 550), (800, 550)],
    [(200, 500), (800, 500)],
    [(200, 450), (700, 450)],
    [(300, 400), (700, 400)],
    [(300, 350), (600, 350)],
    [(100, 250), (900, 250)],
    [(100, 600), (100, 200)],
    [(200, 500), (200, 200)],
    [(300, 400), (300, 200)],
    [(600, 350), (600, 200)],
    [(700, 450), (700, 200)],
    [(800, 550), (800, 200)],
    [(900, 600), (900, 250)],
    [(580, 200), (600, 250)],
    [(620, 200), (600, 250)],
    [(320, 200), (300, 250)],
    [(280, 200), (300, 250)],
    [(680, 200), (700, 250)],
    [(720, 200), (700, 250)],
    [(180, 200), (200, 250)],
    [(220, 200), (200, 250)],
    [(780, 200), (800, 250)],
    [(820, 200), (800, 250)],
    [(80, 200), (100, 250)],
    [(120, 200), (100, 250)],
]

points = [
    (100, 250), (100, 550), (100, 600),
    (200, 250), (200, 450), (200, 500),
    (300, 250), (300, 350), (300, 400),
    (600, 250), (600, 350), (700, 250),
    (700, 400), (700, 450), (800, 250),
    (800, 500), (800, 550), (900, 250),
    (900, 600)
]

gates_entry_positions = [
    (600, 250), # Gate 2
    (300, 250), # Gate 3
    (700, 250), # Gate 4
    (200, 250), # Gate 5
    (800, 250), # Gate 6
    (100, 250), # Gate 7
]

pickup_gates_positions = [
    (600, 200), # Gates 2
    (580, 200),
    (620, 200),
    (300, 200), # Gates 3
    (320, 200),
    (280, 200),
    (700, 200), # Gates 4
    (680, 200),
    (720, 200),
    (200, 200), # Gates 5
    (180, 200),
    (220, 200),
    (800, 200), # Gates 6
    (780, 200), 
    (820, 200),
    (100, 200), # Gates 7
    (80, 200),
    (120, 200),
]

In [12]:
class Path:
    
    def __init__(self, name, points):
        self.name = name  
        self.points = points
    
paths = {
    cluster: Path(f"{cluster}", points) for cluster, points in {
        7: [(900, 600), (100, 600), (100, 550), (100, 250)],
        6: [(900, 600), (100, 600), (100, 550), (800, 550), (800, 500), (800, 250)], 
        5: [(900, 600), (100, 600), (100, 550), (800, 550), (800, 500), (200, 500), (200, 450), (200, 250)], 
        4: [(900, 600), (100, 600), (100, 550), (800, 550), (800, 500), (200, 500), (200, 450), (700, 450), 
            (700, 400), (700, 250)],
        3: [(900, 600), (100, 600), (100, 550), (800, 550), (800, 500), (200, 500), (200, 450), (700, 450), 
            (700, 400), (300, 400), (300, 350), (300, 250)],
        2: [(900, 600), (100, 600), (100, 550), (800, 550), (800, 500), (200, 500), (200, 450), (700, 450), 
            (700, 400), (300, 400), (300, 350), (600, 350), (600, 250)]
    }.items()
}

paths_2 = {
  cluster: Path(f"{cluster}", points) for cluster, points in {
        7: [(900, 250), (900, 600)],
        6: [(900, 250), (900, 600)], 
        5: [(900, 250), (900, 600)], 
        4: [(900, 250), (900, 600)],
        3: [(900, 250), (900, 600)],
        2: [(900, 250), (900, 600)]
    }.items()
}

def generate_random_color():
    return (random.randint(0, 255), random.randint(0, 255), random.randint(0, 255))

cluster_sizes = [2, 3, 4, 5, 6, 7, 8, 9, 10] 

cluster_colors = {size: generate_random_color() for size in cluster_sizes}


flights = ['EK4276', 'AA9359', 'BA3438']

# Assign a unique color to each flight
flight_colors = {flight: generate_random_color() for flight in flights}

PPM = 1100/170 

pixel_speed = ((3*PPM)/60)*10

estimated_times = {
    7: 74.70,
    6: 110.76,
    5: 141.67,
    4: 167.42,
    3: 188.03,
    2: 203.48
}

gate_mapping = {
    1: 'A',
    2: 'B',
    3: 'C'
}


In [14]:
class Bag:  
        
    def __init__(self, bag_id, reservation_id, cluster_size, flight_number, flight_time):
        self.ready = False
        self.path = []
        self.bag_id = bag_id
        self.reservation = reservation_id
        self.cluster_size = cluster_size
        self.position = (1200, 600)
        self.assigned_to_gate = False
        self.gate = None
        self.gate_name = None
        self.speed = pixel_speed
        self.target = None 
        self.color = cluster_colors[cluster_size]
        self.at_gate = False
        self.flight_number = flight_number
        self.flight_time = flight_time
        self.time = None
        self.estimated_time = None
        self.actual_time = None
        self.first_time = True
        self.real_time = None
        self.rand = 0
        self.res_path_assigned = False

    def set_at_gate(self):
        self.at_gate = True

    def set_ready(self):
        self.ready = True   

    def set_path(self):
        numbers = [2, 3, 4, 5, 6, 7]
        weights = [2, 1, 2, 1, 2, 1]
        number = random.choices(numbers, weights=weights, k=1)[0]
        self.rand = number
        self.path = paths[number].points[:]  
        if self.path:
            self.target = self.path.pop(0) 

    
    def set_path2(self, rand):
        self.rand = rand
        self.path = paths[rand].points[:]  
        if self.path:
            self.target = self.path.pop(0) 

    def set_gate(self, pickup_gate):
        self.gate = pickup_gate

    def set_gate_name(self, pickup_gate):
        self.gate_name = str(self.cluster_size) + gate_mapping[pickup_gate.name]

    def append_path(self, path):
        if isinstance(path, Path):  
            self.path.extend(path.points)  
        elif isinstance(path, list):  
            self.path.extend(path)  
        elif isinstance(path, tuple):
            self.path.append(path)
        
        if not self.target and self.path:
            self.target = self.path.pop(0) # Get it moving

    def move(self):
        if self.target:
            x, y = self.position
            tx, ty = self.target
    
            dx = tx - x
            dy = ty - y
            distance = (dx**2 + dy**2) ** 0.5
    
            if distance <= self.speed:
                self.position = self.target  
                if self.path:
                    self.target = self.path.pop(0)  
                else:
                    self.target = None  
            else:
                self.position = (x + (dx / distance) * self.speed, 
                                 y + (dy / distance) * self.speed)


In [16]:
class Gate:
    
    def __init__(self, name, gates, position):
        self.name = name 
        self.gates = gates
        self.position = position
        self.res_queue = queue.Queue() 

    def add_res(self, reservation):
        if not reservation in list(self.res_queue.queue):
            self.res_queue.put(reservation)

    def search(self, bag):
        for i in range(3):
            if(self.gates[i].reservation == bag.reservation):
                return i
        for i in range(3):
            if(self.gates[i].reservation == None):
                return i
        return -1
   

In [18]:
class PickupGate:
    
    def __init__(self, name, position):
        self.name = name 
        self.position = position
        self.reservation = None
        self.load = 0
        self.max_load = 10
        self.bags = []
        self.full = False 
        self.dispense_time = None
    
    def set_reservation(self, reservation):
        if self.reservation is None:  
            self.reservation = reservation
            return True
        else:
            return False

    def add_luggage(self, bag):
        if bag.reservation == self.reservation:
            self.bags.append(bag)
            self.load += 1
            
    def is_full(self):
        if(self.load>0):
            if self.load == self.bags[0].cluster_size:
                return True
        return False

    def clear_luggage(self):
        removed_bags = self.bags if self.bags else []  
        self.reservation = None
        self.load = 0
        self.bags = [] 
        return removed_bags  


In [20]:
class Reservation:
    
    def __init__(self, id, cluster_size, flight_time, flight_number):
        self.id = id
        self.cluster_size = cluster_size
        self.flight_time = flight_time
        self.flight_number = flight_number
        self.time = None

    def __str__(self):
        return f"Reservation ID: {self.id}, Cluster Size: {self.cluster_size}, Time: {self.time}, Flight time: {self.flight_time}, Flight number: {self.flight_number}"
    

In [22]:

def convert_flight_time(flight_time_str):
    flight_time_obj = datetime.strptime(flight_time_str + ":00", '%H:%M:%S')
    return flight_time_obj


In [24]:
gate_entries = []
k = 0
for i in range(6):
    gates = []
    for j in range(3):
        gates.append(PickupGate(j+1, pickup_gates_positions[k]))
        k += 1
    gate_entries.append(Gate(i+2, gates, gates_entry_positions[i]))

reservations = []

conveyor_belt = []

# Group bags by Flight Number
grouped_conveyor_belt = []

for flight_number, group in df.groupby("Flight_Number"):
    flight_bags = []
    
    flight_time = group.iloc[0]["Flight_Arrival_Time"]
    flight_start_time = convert_flight_time(flight_time) # Convert to milliseconds
    
    for reservation_id, res_group in group.groupby("Reservation_ID"):
        cluster_size = int(res_group.iloc[0]['Cluster_Size'])  # Get cluster size

        for _, row in res_group.iterrows():
            real_bag_id = row["Bag_ID"]  # Adjust this column name if different
            flight_bags.append(Bag(real_bag_id, reservation_id, cluster_size, flight_number, flight_time))

    # Shuffle bags within this flight
    random.shuffle(flight_bags)

    bag_time = flight_start_time  # Initialize the timestamp for the first bag

    for bag in flight_bags:
        # Assign timestamp after shuffling
        bag.time = datetime.combine(date.today(), bag_time.time())
        bag.estimated_time = (bag_time + timedelta(seconds=estimated_times[bag.cluster_size])).time()
        bag_time += timedelta(seconds=3)/10 

    # Optionally: store flight index to simulate 12-min delay later
    for bag in flight_bags:
        bag.flight_index = len(grouped_conveyor_belt)

    grouped_conveyor_belt.append(flight_bags)

for flight_group in grouped_conveyor_belt:
    conveyor_belt.extend(flight_group)

conveyor_belt = sorted(conveyor_belt, key=lambda bag: bag.time)

reservations_info = {}

for bag in conveyor_belt:
    if bag.reservation not in reservations_info:
        reservations_info[bag.reservation] = (bag.cluster_size, bag.flight_time, bag.flight_number)

reservations_f = {
    res_id: Reservation(res_id, cluster_size, flight_time, flight_number)
    for res_id, (cluster_size, flight_time, flight_number) in reservations_info.items()
}

reservations_c = []

In [26]:
pygame.init()

width, height = 1600, 800
screen = pygame.display.set_mode((width, height))
pygame.display.set_caption("Random Visualization")

WHITE = (255, 255, 255)
BLACK = (0, 0, 0)
RED = (255, 0, 0)

In [27]:
scanned_bags = []
i = 0

clock = pygame.time.Clock()
running = True
last_spawn_time = 0  
spawn_interval = 2000 
title_font = pygame.font.SysFont('Arial', 28, bold=True)
font = pygame.font.Font(None, 20)
bag_start_times = []  
start_time = pygame.time.get_ticks()  

while running:
    
    screen.fill(WHITE) 
    
    elapsed_ms = (pygame.time.get_ticks() - start_time)*10
    elapsed_seconds = elapsed_ms // 1000
    minutes = elapsed_seconds // 60
    seconds = elapsed_seconds % 60
    
    time_text = font.render(f"Time: {minutes:02}:{seconds:02}", True, BLACK)
    screen.blit(time_text, (10, 10))

    title_text = title_font.render("Random", True, BLACK)
    screen.blit(title_text, (width // 2 - title_text.get_width() // 2, 20))
    '''
    flight_text = font.render(f"Flight: {read_bag.flight_number}", True, BLACK)
    screen.blit(flight_text, (1000, 10))   ''' 

    for belt in conveyor_belts:
        pygame.draw.line(screen, BLACK, belt[0], belt[1], 3)

    for point in points:
        pygame.draw.circle(screen, BLACK, point, 5)

    for gate in gates_entry_positions:
        pygame.draw.rect(screen, BLACK, (gate[0] - 5, gate[1] - 5, 10, 10))

    for pickup in pickup_gates_positions:
        pygame.draw.rect(screen, BLACK, (pickup[0] - 5, pickup[1] - 5, 10, 10))

    for gate_group in gate_entries: 
        for gate in gate_group.gates: 
            load_text = font.render(f"{gate.load}", True, BLACK)    
            screen.blit(load_text, (gate.position[0]-5, 177))
        
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False


    current_ticks = pygame.time.get_ticks()
    
    current_time = convert_flight_time(conveyor_belt[0].flight_time) + timedelta(milliseconds=current_ticks*10)
    current_time = current_time.time()
            
    more_items = len(conveyor_belt)-1

    if i < len(conveyor_belt): 
        read_bag = conveyor_belt[i] 
        if current_time >= read_bag.time.time():
        
            read_bag.real_time = datetime.now()
            scanned_bags.append(read_bag)
            
            read_bag.set_ready()  

            
            bag_start_times.append(read_bag.time.time())

            if not read_bag.res_path_assigned:
                
                read_bag.res_path_assigned = True
                read_bag.set_path()
    
                rand = read_bag.rand
                        
            
                gate = gate_entries[rand-2].search(read_bag)
                
                if gate == -1:
                    gate_entries[rand-2].add_res(read_bag.reservation)
                else:
                    gate_entries[rand-2].gates[gate].set_reservation(read_bag.reservation)
                    read_bag.append_path(gate_entries[rand-2].gates[gate].position)
                    read_bag.set_gate(gate_entries[rand-2].gates[gate])
                    read_bag.set_gate_name(gate_entries[rand-2].gates[gate])
                    read_bag.assigned_to_gate = True

                for bag_2 in conveyor_belt:
                    if bag_2.reservation == read_bag.reservation:
                        bag_2.set_path2(rand)
                        bag_2.res_path_assigned = True
                        bag_2.rand = rand
                        if gate != -1:
                            bag_2.append_path(gate_entries[rand-2].gates[gate].position)
                            bag_2.set_gate(gate_entries[rand-2].gates[gate])
                            bag_2.set_gate_name(gate_entries[rand-2].gates[gate])
                            bag_2.assigned_to_gate = True 
                
            
            i+=1
    
    still_moving = False 
    
    j=0
    for bag in scanned_bags:
        
        if bag.cluster_size<8:            
            if bag.assigned_to_gate and bag.position == gate_entries[bag.rand-2].gates[bag.gate.name-1].position and bag.at_gate:
                pygame.draw.circle(screen, bag.color, bag.position, 8)
                if bag.first_time:
                    elapsed = datetime.now() - bag.real_time # both must be datetime objects
                    bag.actual_time = (elapsed*10 + bag.time).time()
                    bag.first_time = False
            elif bag.assigned_to_gate and bag.position == gate_entries[bag.rand-2].gates[bag.gate.name-1].position:
                gate_entries[bag.rand-2].gates[bag.gate.name-1].add_luggage(bag)
                bag.set_at_gate()
                pygame.draw.circle(screen, bag.color, bag.position, 8)
            elif not bag.assigned_to_gate and bag.position == gate_entries[bag.rand-2].position:
                bag.append_path(paths_2[bag.rand])
                bag.append_path(paths[bag.rand])
                if bag.path: 
                    bag.move()
                    pygame.draw.circle(screen, bag.color, bag.position, 8)
            elif current_time >= bag_start_times[j]:
                bag.move()
                pygame.draw.circle(screen, bag.color, bag.position, 8)
                still_moving = True 

        else:
            if bag.assigned_to_gate and bag.position == gate_entries[bag.cluster_size%7-1].gates[bag.gate.name-1].position and bag.at_gate:
                pygame.draw.circle(screen, bag.color, bag.position, 8)
            elif bag.assigned_to_gate and bag.position == gate_entries[bag.cluster_size%7-1].gates[bag.gate.name-1].position:
                gate_entries[bag.cluster_size%7-1].gates[bag.gate.name-1].add_luggage(bag)
                bag.set_at_gate()
                pygame.draw.circle(screen, bag.color, bag.position, 8)
            elif not bag.assigned_to_gate and bag.position == gate_entries[bag.cluster_size%7-1].position:
                bag.append_path(paths_2[bag.cluster_size%7-1])
                bag.append_path(paths[bag.cluster_size%7-1])
                if bag.path: 
                    bag.move()
                    pygame.draw.circle(screen, bag.color, bag.position, 8)
            elif current_time >= bag_start_times[j]:
                bag.move()
                pygame.draw.circle(screen, bag.color, bag.position, 8)
                still_moving = True 

        
        for gate_group in gate_entries:
            for gate in gate_group.gates:
                if gate.is_full():
                    res_id = gate.reservation
                    if res_id in reservations_f:  
                        reservations_f[res_id].time = current_time
                        reservations_c.append(reservations_f.pop(res_id))
         

        for gate_group in gate_entries: 
            for gate in gate_group.gates: 
                if gate.is_full() and gate.dispense_time is None:
                    wait_seconds = random.randint(0, 70)/10
                    gate.dispense_time = datetime.combine(datetime.today(), current_time) + timedelta(seconds=wait_seconds)
                            

        for gate_group in gate_entries: 
            for gate in gate_group.gates: 
                if gate.dispense_time and current_time >= gate.dispense_time.time():
                    removed_bags = gate.clear_luggage()  
                    scanned_bags = [bag for bag in scanned_bags if bag not in removed_bags]  
                    print(f"Gate {gate_group.name} was emptied, and {len(removed_bags)} bags disappeared at {current_time}!")
                    gate.dispense_time = None  


        for gate_group in gate_entries: 
            for gate in gate_group.gates: 
                if gate.reservation == None: 
                    if not gate_group.res_queue.empty():
                        res = gate_group.res_queue.get() 
                        gate.set_reservation(res)

        for gate_group in gate_entries: 
            for gate in gate_group.gates: 
                if not bag.assigned_to_gate and bag.reservation == gate.reservation:
                    bag.append_path(gate.position)
                    bag.set_gate(gate)
                    bag.set_gate_name(gate)
                    bag.assigned_to_gate = True

        j+=1

    pygame.display.update()
    clock.tick(60) 

    
    if i >= len(conveyor_belt) and not still_moving:
        running = False  
        pygame.quit()



2025-05-13 21:49:15.019 python[91940:5375380] +[IMKClient subclass]: chose IMKClient_Modern
2025-05-13 21:49:15.019 python[91940:5375380] +[IMKInputSession subclass]: chose IMKInputSession_Modern


Gate 7 was emptied, and 5 bags disappeared at 12:46:02.780000!
Gate 7 was emptied, and 7 bags disappeared at 12:46:22.690000!
Gate 7 was emptied, and 3 bags disappeared at 12:46:30.990000!
Gate 6 was emptied, and 6 bags disappeared at 12:47:04.130000!
Gate 6 was emptied, and 6 bags disappeared at 12:47:04.640000!
Gate 6 was emptied, and 7 bags disappeared at 12:47:11.590000!
Gate 4 was emptied, and 7 bags disappeared at 12:47:30.840000!
Gate 5 was emptied, and 5 bags disappeared at 12:47:33.140000!
Gate 4 was emptied, and 7 bags disappeared at 12:47:33.480000!
Gate 5 was emptied, and 6 bags disappeared at 12:47:41.330000!
Gate 5 was emptied, and 3 bags disappeared at 12:47:45.630000!
Gate 4 was emptied, and 6 bags disappeared at 12:47:51.330000!
Gate 7 was emptied, and 3 bags disappeared at 12:48:02.630000!
Gate 3 was emptied, and 6 bags disappeared at 12:48:08.690000!
Gate 3 was emptied, and 3 bags disappeared at 12:48:17.330000!
Gate 7 was emptied, and 6 bags disappeared at 12:48:24.

In [28]:
bag_time_map = {}
for bag in conveyor_belt:
    # Make sure these are Python strings "HH:MM:SS"
    est = bag.estimated_time
    act = getattr(bag, "actual_time", None)
    # If est/act are datetime.time or timedelta, convert them now:
    if hasattr(est, "strftime"):
        est = est.strftime("%H:%M:%S")
    if hasattr(act, "strftime"):
        act = act.strftime("%H:%M:%S")
    bag_time_map[bag.bag_id] = (est, act)

'''
s3 = boto3.client("s3")
bucket = "trackage.1"
key = "database.csv"

obj = s3.get_object(Bucket=bucket, Key=key)
df = pd.read_csv(obj["Body"], on_bad_lines="skip")
'''
gate_name_map = {str(bag.bag_id): bag.gate_name for bag in conveyor_belt}
df["Bag_ID"] = df["Bag_ID"].astype(str)
df["Pickup_Gate"] = df["Bag_ID"].map(gate_name_map)


df["Estimated_Arrival_Time"] = df["Bag_ID"].map(lambda bid: bag_time_map.get(bid, ("", ""))[0])
df["Actual_Arrival_Time"]    = df["Bag_ID"].map(lambda bid: bag_time_map.get(bid, ("", ""))[1])
df["Flight_Arrival_Time"] = (
    pd.to_datetime(df["Flight_Arrival_Time"], format="%H:%M", errors="coerce")
      .subtract(pd.Timedelta(minutes=15))
      .dt.strftime("%H:%M")
)


out_tmp     = "/tmp/luggage_dataset.csv"
out_desktop = os.path.join(os.path.expanduser("~"), "Desktop", "luggage_dataset.csv")

df.to_csv(out_tmp, index=False)
df.to_csv(out_desktop, index=False)
'''
s3.upload_file(out_tmp, bucket, key)
print("Saved to:", out_desktop)
print(f"Uploaded back to s3://{bucket}/{key}")'''

'\ns3.upload_file(out_tmp, bucket, key)\nprint("Saved to:", out_desktop)\nprint(f"Uploaded back to s3://{bucket}/{key}")'

In [ ]:
for res in reservations_c:
    flight_time_obj = datetime.strptime(res.flight_time, "%H:%M" if len(res.flight_time) == 5 else "%H:%M:%S").time()

    served_dt = datetime.combine(datetime.today(), res.time)
    flight_dt = datetime.combine(datetime.today(), flight_time_obj)
    res.time = served_dt - flight_dt

    print(res)    